In [6]:
## disc loading 
# gross weight in kg  disc loading we get in kg/m2

# W0 = gross weight
# DL = discloading 
def get_disc_lodaing(W0):
    a4_kg = 2.12
    a5_kg = 0.57 #when DL is given in kg/m2 and W0 is given in kg.
    
    a4= a4_kg
    a5= a5_kg

    
    DL = a4*(W0**(1/3) - a5)
    return DL
print(get_disc_lodaing(2000))
        
        

25.50192625777131


In [7]:
# D is Main rotor diameter Wo is in kg and V in m/s   retunrs in m
def get_main_rotor_daimeter(W0,V_max):
    D = 9.133*((W0**0.380)/(V_max**0.515))
    return D


In [8]:
#Main rotor solidity(blade area/ disc area) Nb number of blades , C chord 

def get_main_rotor_chord(Nb, W0):
    c = 0.0108*((W0**0.539)/(Nb**0.714))
    return c
    
    

In [9]:
# vtip  in m/s
def get_rotor_tip_speed(D):
    V_tip = 140/(D**0.171)
    ang_vel  = V_tip*2/D
    return V_tip,ang_vel

In [10]:
def get_tail_rotor_daimeter(W0):
    D_tr = 0.0886*(W0**0.393)
    return D_tr

In [11]:
# rotor to rotor distance from main to tail rotor
#a_mt = tail rotor arm
def get_tail_rotor_arm(D):
    a_mt = 0.5107*(D**1.061)
    return a_mt

In [12]:
# Tail rotor tip speed and angular velocity  V_tip_tail
def get_rotor_tip_speed(D_tr):
    V_tip = 182/(D_tr**0.172)
    ang_vel  = V_tip*2/D_tr
    return V_tip,ang_vel
# Tail rotor solidity  Nb_tr -> number of tail rotor blades 
def get_tail_rotor_chord(W0,Nb_tr):
    c_tr = 0.0058*(W0**0.506)/(Nb_tr**0.720)
    return c_tr



In [13]:
# Horizontal tail arm distance between hori tail center and main rotor 
def get_htail_arm(W0):
    a_ht = 0.4247*(W0**0.327)
    return a_ht
def get_htail_surf_A(W0):
    S_ht = 0.0021*(W0**0.758)
    return S_ht
# Vertical tail surface arm distance between ver tail center and main rotor 
def get_vtail_arm(D):
    a_vt = 0.5914*(D**0.995)
    return a_vt
def get_vail_surf_A(D):
    S_vt =  0.5914*(D**0.995)
    return S_vt
def avg_vtail_chord(D_tr):
    if D_tr <3.5:
        c_vt = 0.1605*(D_tr**1.745)
    else:
        c_vt =  0.297*(D_tr**1.06)
    return c_vt




In [14]:
# Airframe
def get_airframe(D):
    FL = 0.824*(D**1.056) #fuselage length nose to vtail
    L_rt= 1.09*(D**1.03) # main rotor start to tail rotor end
    FH = 0.642*(D**0.677) #  hight of the copter 
    FW = 0.436*(D**0.697) # width including landing gear
    return FL,L_rt,FH,FW

    

In [15]:
# weight 
# W0 = Wu(useful weight) + We(empty weight)
# Wu = W_pl(payload) + Wf(fuel) + Wc(crew)
# Rg = range
# rho_f = fuel density kg/l
def get_weights(W0,Rg,W_pl,Wc,rho_f):
    We = 0.4854*(W0**1.015)
    Wf = (0.0038*(W0**0.976)*(Rg**0.650))*rho_f
    Wu_possible = 0.4709*(W0**0.99)
    Wu = Wf +W_pl+Wc
    msg = "all good"
    if (Wu-Wu_possible)/Wu > 0.1:
        msg= "not enough fuel change W0 or reduce Rg"
        
        
    return Wu,We,msg


In [62]:
def get_gross_weight_iterative(W_pl, Wc, rho_f, Rg, tolerance=1e-3, max_iterations=1000):
    """
    Calculates the gross weight iteratively until it converges.

    Args:
        W_pl (float): Payload weight.
        Wc (float): Crew weight.
        rho_f (float): Fuel density.
        Rg (float): Range.
        tolerance (float): The convergence tolerance.
        max_iterations (int): The maximum number of iterations.

    Returns:
        float: The final converged gross weight.
    """
    W0_guess = 5010
    W0_new = 0
    iteration = 0

    while abs(W0_new - W0_guess) > tolerance and iteration < max_iterations:
        if iteration > 0:
            W0_guess = W0_new

        Wf = (0.0038 * (W0_guess**0.976) * (Rg**0.650)) * rho_f
        Wu = Wf + W_pl + Wc
        W0_new = (Wu / 0.4709)**(1 / 0.99)
        # We = 0.4854*(W0_new**1.015)
        # W0_new = Wu+We
        iteration += 1
        print(f"itr:{iteration} residual :{W0_new - W0_guess} ")
    if(iteration>max_iterations):
        print(f"Warning: Maximum iterations ({max_iterations}) reached before convergence. reduce range or reduce payload")
    else:
        print(f"Convergence achieved in {iteration} iterations.")
        print(f"The final converged gross weight is: {W0_new:.2f} lbs")

    return W0_new
Wo_new = get_gross_weight_iterative(
    W_pl=1400,# Target payload (kg)
    Wc=100,    # Target crew weight (kg)
    Rg=439 ,    # Target range (km)
    rho_f=0.8, 
)
print(Wo_new)
     
    
    

itr:1 residual :-43.52083952628709 
itr:2 residual :-12.83107042318261 
itr:3 residual :-3.783382447322765 
itr:4 residual :-1.1156113412107516 
itr:5 residual :-0.3289653225683651 
itr:6 residual :-0.09700377894478152 
itr:7 residual :-0.02860405340561556 
itr:8 residual :-0.008434641399617249 
itr:9 residual :-0.002487171314896841 
itr:10 residual :-0.0007334065567192738 
Convergence achieved in 10 iterations.
The final converged gross weight is: 4948.28 lbs
4948.282867887807


In [17]:
# speed - never exceed speed and long range speed 
def speeds(V_max):
    V_ne = 0.8215*(V_max**1.056)
    V_lr = 0.5475*(V_max**1.0899)
    return V_ne,V_lr
    
    

In [18]:
#power  p_to -takeoff_power
# T_to is the take-off transmission
def get_power(W0,V_max):
    P_to= 0.0764*(W0**1.1455)
    T_to =0.0366*(W0**1.2107)
    P_mc= 0.00126*(W0**0.9876)*(V_max**0.9760)
    T_mc =0.000141*(W0**0.9771)*(V_max**1.3393)
    
    
    return P_to,T_to,P_mc,T_mc

In [47]:
def calculate_helicopter_parameters(W0, V_max, Nb, Nb_tr, W_pl, Wc, Rg, rho_f):
    """
    Calculates key helicopter parameters based on given inputs.

    Args:
        W0 (float): Gross weight in kg.
        V_max (float): Maximum speed in m/s.
        Nb (int): Number of main rotor blades.
        Nb_tr (int): Number of tail rotor blades.
        W_pl (float): Payload weight in kg.
        Wc (float): Crew weight in kg.
        Rg (float): Range in km.
        rho_f (float): Fuel density in kg/L.
    
    Returns:
        A dictionary containing the calculated parameters.
    """
    
    # Main rotor and tail rotor calculations
    DL = get_disc_lodaing(W0)
    D = get_main_rotor_daimeter(W0, V_max)
    c = get_main_rotor_chord(Nb, W0)
    V_tip_main, ang_vel_main = get_rotor_tip_speed(D)
    
    D_tr = get_tail_rotor_daimeter(W0)
    a_mt = get_tail_rotor_arm(D)
    V_tip_tail, ang_vel_tail = get_rotor_tip_speed(D_tr)
    c_tr = get_tail_rotor_chord(W0, Nb_tr)

    # Airframe and tail dimensions
    FL, L_rt, FH, FW = get_airframe(D)
    a_ht = get_htail_arm(W0)
    S_ht = get_htail_surf_A(W0)
    a_vt = get_vtail_arm(D)
    S_vt = get_vail_surf_A(D)
    c_vt = avg_vtail_chord(D_tr)

    # Weight and fuel calculations
    Wu, We,_ = get_weights(W0, Rg, W_pl, Wc, rho_f)
    
    # Speed and power
    V_ne, V_lr = speeds(V_max)
    P_to, T_to, P_mc, T_mc = get_power(W0, V_max)

    # Organize results into a dictionary
    results = {
        "Main Rotor": {
            "Disc Loading (kg/m^2)": DL,
            "Main Rotor Diameter (m)": D,
            "Main Rotor Chord (m)": c,
            "Main Rotor Tip Speed (m/s)": V_tip_main,
            "Main Rotor Angular Velocity (rad/s)": ang_vel_main,
        },
        "Tail Rotor": {
            "Tail Rotor Diameter (m)": D_tr,
            "Tail Rotor Arm (m)": a_mt,
            "Tail Rotor Tip Speed (m/s)": V_tip_tail,
            "Tail Rotor Angular Velocity (rad/s)": ang_vel_tail,
            "Tail Rotor Chord (m)": c_tr,
        },
        "Airframe and Tail": {
            "Fuselage Length (m)": FL,
            "Rotor-to-Tail-End Length (m)": L_rt,
            "Helicopter Height (m)": FH,
            "Helicopter Width (m)": FW,
            "Horizontal Tail Arm (m)": a_ht,
            "Horizontal Tail Surface Area (m^2)": S_ht,
            "Vertical Tail Arm (m)": a_vt,
            "Vertical Tail Surface Area (m^2)": S_vt,
            "Average Vertical Tail Chord (m)": c_vt,
        },
        "Weights": {
            "Gross takeoff weight": (W0,Wu+We),
            "Useful Weight (kg)": Wu,
            "Empty Weight (kg)": We,
        },
        "Performance": {
            "Never Exceed Speed (m/s)": V_ne,
            "Long Range Speed (m/s)": V_lr,
        },
        "Power and Transmission": {
            "Take-off Power (P_to)(kW)": P_to,
            "Take-off Transmission (T_to)(kW)": T_to,
            "Main continuous Power (P_mc)(kW)": P_mc,
            "Main continuous Transmission (T_mc)(kW)": T_mc,
        }
    }

    return results

# Example Usage
# W0 = Gross weight (kg), V_max = Max speed (m/s), Nb = number of main rotor blades, 
# Nb_tr = number of tail rotor blades, W_pl = payload (kg), Wc = crew (kg),
# Rg = range (km), rho_f = fuel density (kg/L)
example_parameters = calculate_helicopter_parameters(
    W0=5100, 
    V_max=200, 
    Nb=2, 
    Nb_tr=2, 
    W_pl=2117, 
    Wc=150, 
    Rg=439 , 
    rho_f=0.8
)

print(example_parameters)

{'Main Rotor': {'Disc Loading (kg/m^2)': 35.28317315939789, 'Main Rotor Diameter (m)': 15.291837587800744, 'Main Rotor Chord (m)': 0.6559468691380018, 'Main Rotor Tip Speed (m/s)': 113.85295934364989, 'Main Rotor Angular Velocity (rad/s)': 14.890683829192316}, 'Tail Rotor': {'Tail Rotor Diameter (m)': 2.5381004207247666, 'Tail Rotor Arm (m)': 9.223113963447688, 'Tail Rotor Tip Speed (m/s)': 155.05860561373333, 'Tail Rotor Angular Velocity (rad/s)': 122.18476806323969, 'Tail Rotor Chord (m)': 0.2646774280306076}, 'Airframe and Tail': {'Fuselage Length (m)': 14.679681413864436, 'Rotor-to-Tail-End Length (m)': 18.089225180889336, 'Helicopter Height (m)': 4.068311789850657, 'Helicopter Width (m)': 2.91779572735103, 'Horizontal Tail Arm (m)': 6.925604848867887, 'Horizontal Tail Surface Area (m^2)': 1.3569301373027913, 'Vertical Tail Arm (m)': 8.921105978339247, 'Vertical Tail Surface Area (m^2)': 8.921105978339247, 'Average Vertical Tail Chord (m)': 0.8153477829896456}, 'Weights': {'Gross t

In [ ]:
# # The original helper functions from your notebook go here.
# # (e.g., get_disc_lodaing, get_main_rotor_daimeter, get_weights, etc.)

# # Example of an iterative main function
def iterative_helicopter_modeler(W_pl_target, Wc_target, Rg_target, rho_f, V_max, Nb, Nb_tr, W0_initial_guess, tolerance=0.01):
    """
    Iteratively finds the optimal gross weight (W0) for a helicopter 
    based on a target useful weight.
    
    Args:
        W_pl_target (float): Target payload weight in kg.
        Wc_target (float): Target crew weight in kg.
        Rg_target (float): Target range in km.
        rho_f (float): Fuel density in kg/L.
        V_max (float): Maximum speed in m/s.
        Nb (int): Number of main rotor blades.
        Nb_tr (int): Number of tail rotor blades.
        W0_initial_guess (float): Initial guess for gross weight in kg.
        tolerance (float): Convergence tolerance (e.g., 0.01 for 1%).
        
    Returns:
        A dictionary with all calculated parameters for the final W0.
    """
    
    W0 = W0_initial_guess
    
    # Calculate the total mission useful weight
    W0 = get_gross_weight_iterative(W_pl_target,Wc_target,rho_f,Rg_target)
    
        
    # Use the final W0 to calculate all other parameters
    final_parameters = calculate_helicopter_parameters(W0, V_max, Nb, Nb_tr, W_pl_target, Wc_target, Rg_target, rho_f)
    return final_parameters

# Define a function to calculate all parameters
    # This is the same function from the previous response.
    # It takes a final W0 and calculates all the other parameters.
    # ... (function body)
    
# Example of how to run the iterative modeler
final_design = iterative_helicopter_modeler(
    W_pl_target=1500,# Target payload (kg)
    Wc_target=100,    # Target crew weight (kg)
    Rg_target=439 ,    # Target range (km)
    rho_f=0.8,        # Fuel density (kg/L)
    V_max=200,         # Maximum speed (m/s)
    Nb=2,             # Number of main rotor blades
    Nb_tr=2,          # Number of tail rotor blades
    W0_initial_guess=4000, # Initial guess for gross weight
)
# print(final_design)

# You can now print the results of the final, converged design
print("\nFinal Helicopter Design Parameters:")
for section, params in final_design.items():
    print(f"--- {section} ---")
    for key, value in params.items():
        print(f"{key}: {value}")

itr:1 residual :190.0922505917988 
itr:2 residual :56.042554213468975 
itr:3 residual :16.514013151409927 
itr:4 residual :4.865461267278988 
itr:5 residual :1.433431216308236 
itr:6 residual :0.422303069721238 
itr:7 residual :0.12441421719540813 
itr:8 residual :0.03665348815957259 
itr:9 residual :0.010798426363180624 
itr:10 residual :0.0031813070654607145 
itr:11 residual :0.0009372397444167291 
Convergence achieved in 11 iterations.
The final converged gross weight is: 5279.55 lbs

Final Helicopter Design Parameters:
--- Main Rotor ---
Disc Loading (kg/m^2): 35.70647370575885
Main Rotor Diameter (m): 15.494219692472115
Main Rotor Chord (m): 0.6682945110901293
Main Rotor Tip Speed (m/s): 113.595779998009
Main Rotor Angular Velocity (rad/s): 14.662988166251397
--- Tail Rotor ---
Tail Rotor Diameter (m): 2.5728482817115417
Tail Rotor Arm (m): 9.352676648384334
Tail Rotor Tip Speed (m/s): 154.69637946508277
Tail Rotor Angular Velocity (rad/s): 120.25301341295861
Tail Rotor Chord (m):

In [81]:
import numpy as np
from scipy.optimize import fsolve

def solve_gross_weight_with_solver(W_pl_target, Rg, rho_f, Wc=0):
    """
    Solves the gross weight equation using a numerical solver.

    Args:
        W_pl_target (float): The target payload weight in kg.
        Rg (float): The mission range in km.
        rho_f (float): The fuel density in kg/L.
        Wc (float): Crew weight in kg.

    Returns:
        float: The final calculated gross weight (W0) in kg.
    """
    # Define the equation as a function that returns the difference from zero.
    # We want to find W0 such that:
    # (W0 - W_empty - W_fuel) - (W_pl + Wc) = 0
    def equation_to_solve(W0):
        W_empty = 0.4854 * (W0**1.015)
        W_fuel = (0.0038 * (W0**0.976) * (Rg**0.650)) * rho_f
        return (W0 - W_empty - W_fuel) - (W_pl_target + Wc)

    # Use fsolve to find the root (the value of W0 where the equation is zero)
    W0_initial_guess = 5000
    W0_solution = fsolve(equation_to_solve, W0_initial_guess)
    
    return W0_solution[0]

# Example usage with values from the image:
payload_weight = 15*110
range_km = 439
fuel_density = 0.8
crew_weight = 0

final_W0 = solve_gross_weight_with_solver(
    W_pl_target=payload_weight,
    Rg=range_km,
    rho_f=fuel_density,
    Wc=crew_weight
)

print(f"The required gross weight is {final_W0:.2f} kg.")

The required gross weight is 5173.32 kg.


In [ ]:
def iterative_gross_weight_calculator(self,W0_guess, tolerance=1e-3, max_iterations=1000):
    """
    Iteratively calculates the gross weight (W0) until it converges,
    based on useful weight requirements.
    
    This function is kept separate as a utility since it doesn't depend on
    the full set of HelicopterDesigner parameters.
    """

    W0_new = 0
    iteration = 0

    while abs(W0_new - W0_guess) > tolerance and iteration < max_iterations:
        if iteration > 0:
            W0_guess = W0_new
        
        # Calculate required useful weight components
        Wf = (0.0038 * (W0_guess**0.976) * (self.Rg**0.650)) * self.rho_f
        Wu = Wf + self.W_pl + self.Wc
        
        # Recalculate W0 based on the useful weight approximation
        W0_new = (Wu / 0.4709)**(1 / 0.99)
        
        iteration += 1
        print(f"Iteration: {iteration}, Gross Weight Guess: {W0_guess:.2f} kg, New W0: {W0_new:.2f} kg")

    if iteration >= max_iterations:
        print(f"Warning: Maximum iterations ({max_iterations}) reached before convergence. "
            "Consider reducing range or payload.")
    else:
        print(f"\nConvergence achieved in {iteration} iterations.")
        print(f"The final converged gross weight is: {W0_new:.2f} kg")
        
    self.W0 = W0_new